### Quantum Sampling Regression - QSR
From paper @misc{rivero2020optimalquantumsamplingregression,
      title={An optimal quantum sampling regression algorithm for variational eigensolving in the low qubit number regime}, 
      author={Pedro Rivero and Ian C. Cloët and Zack Sullivan},
      year={2020},
      eprint={2012.02338},
      archivePrefix={arXiv},
      primaryClass={quant-ph},
      url={https://arxiv.org/abs/2012.02338}, 
}

In [21]:
from qiskit import QuantumCircuit
import numpy as np

from qiskit.primitives import StatevectorEstimator
estimator = StatevectorEstimator()

def cost_func_qsr(parameters, ansatz, observable, estimator):
    """Return estimate of energy from estimator

    Parameters:
        params (ndarray): Array of ansatz parameters
        ansatz (QuantumCircuit): Parameterized ansatz circuit
        hamiltonian (SparsePauliOp): Operator representation of Hamiltonian
        estimator (Estimator): Estimator primitive instance

    Returns:
        float: Energy estimate
    """

    estimator_job = estimator.run([(ansatz, observable, [parameters])])
    estimator_result = estimator_job.result()[0]

    cost = estimator_result.data.evs[0]
    return cost




In [30]:
def estimate_bandwidth_from_circuit(circuit):
    """
    Estimate the Fourier bandwidth S from a manually defined circuit.
    Works directly without decomposition.
    """
    S = 0
    entanglement_present = False

    for instr in circuit.data:
        name = instr.operation.name.lower()

        if name in ['rx', 'ry', 'rz']:
            S += 1
        elif name in ['cx', 'cz', 'iswap', 'cy']:
            entanglement_present = True

    # Debug print
    print(f"[DEBUG] Rotations: {S}, Entanglement: {entanglement_present}")

    if entanglement_present:
        S = max(2*S, S+1)

    return max(S, 1)


In [31]:
def sampler_qsr(parameters_list, ansatz, observable, estimator):
    """
    Sample the cost function over a list of parameter vectors.
    
    Args:
        parameters_list: list of parameter arrays (each array shape = (N_parameters,))
        ansatz: QuantumCircuit, parameterized ansatz
        observable: Hamiltonian (SparsePauliOp)
        estimator: Estimator primitive (e.g., StatevectorEstimator)
        
    Returns:
        cost_sampled: list of cost function values corresponding to the parameter list
    """
    
    cost_sampled = []
    
    for params in parameters_list:
        # params must be an array of all θ values [θ₁, θ₂, ..., θₙ]
        cost_value = cost_func_qsr(params, ansatz, observable, estimator)
        cost_sampled.append(cost_value)

    return cost_sampled


In [32]:
import numpy as np
import itertools

def build_F_matrix(theta_samples, max_harmonic):
    """
    Build the measurement matrix F for QSR with multiple parameters.
    
    Args:
        theta_samples: array-like, shape (N_samples, N_parameters)
        max_harmonic: int, maximum Fourier mode per parameter
        
    Returns:
        F: (N_samples, N_basis_functions) numpy array
        all_freqs: list of frequency tuples
    """
    theta_samples = np.array(theta_samples)
    N_samples, N_parameters = theta_samples.shape
    
    # Generate all combinations of frequencies for each parameter
    freq_range = list(range(0, max_harmonic + 1))  # 0,1,...,S
    all_freqs = list(itertools.product(freq_range, repeat=N_parameters))
    
    # Remove the trivial case where all frequencies are 0
    all_freqs.remove((0,) * N_parameters)
    
    # Initialize the F matrix
    N_basis = 1 + 2 * len(all_freqs)  # 1 constant + 2 (cos and sin) for each frequency combo
    F = np.zeros((N_samples, N_basis))
    
    # First column = constant term
    F[:, 0] = 1.0

    # Fill the F matrix
    for i, freqs in enumerate(all_freqs):
        # Compute the dot product k · theta for each sample
        argument = np.sum(freqs * theta_samples, axis=1)  # shape (N_samples,)
        
        F[:, 2*i + 1] = np.cos(argument)  # cosine term
        F[:, 2*i + 2] = np.sin(argument)  # sine term
        
    return F, all_freqs



def solve_fourier_coefficients(theta_samples, h_samples, max_harmonic):
    """
    Solve for Fourier coefficients c in multivariate case.
    
    Args:
        theta_samples: array-like, shape (N_samples, N_parameters)
        h_samples: array-like, shape (N_samples,)
        max_harmonic: int, maximum Fourier harmonic S
        
    Returns:
        c: (N_basis_functions,) numpy array of Fourier coefficients
        all_freqs: list of frequency tuples used
    """
    F, all_freqs = build_F_matrix(theta_samples, max_harmonic)
    h_samples = np.array(h_samples)
    
    # Solve least squares: minimize || F c - h ||
    c, residuals, rank, s = np.linalg.lstsq(F, h_samples, rcond=None)
    
    return c, all_freqs


In [33]:
import numpy as np
from scipy.optimize import minimize

def reconstructed_h(theta, coefficients, all_freqs):
    """
    Reconstruct the function h(theta) from Fourier coefficients and frequency list.
    
    Args:
        theta: array-like of θ values
        coefficients: array of Fourier coefficients
        all_freqs: list of frequency tuples
        
    Returns:
        h(theta)
    """
    theta = np.array(theta)
    h = coefficients[0]  # constant term

    for i, freqs in enumerate(all_freqs):
        argument = np.dot(freqs, theta)  # dot product
        h += coefficients[2*i + 1] * np.cos(argument)
        h += coefficients[2*i + 2] * np.sin(argument)
    
    return h

def find_global_minimum(coefficients, all_freqs):
    """
    Find the global minimum of reconstructed h(theta) for multiple parameters.
    
    Args:
        coefficients: array of Fourier coefficients
        all_freqs: list of frequency tuples
        
    Returns:
        (theta_min, h_min)
    """
    N_parameters = len(all_freqs[0])  # inferred from freq tuples
    
    # Initial guess: center of the domain
    x0 = np.full(N_parameters, np.pi)

    # Bounds: θ ∈ [0, 2π] for each parameter
    bounds = [(0, 2*np.pi) for _ in range(N_parameters)]

    # Minimize
    result = minimize(
        lambda theta: reconstructed_h(theta, coefficients, all_freqs),
        x0=x0,
        bounds=bounds,
        method='L-BFGS-B'  # Good for bounded smooth problems
    )
    
    theta_min = result.x
    h_min = result.fun
    return theta_min, h_min


In [34]:
def qsr_main(ansatz, observable):
    

SyntaxError: incomplete input (89219631.py, line 2)

### Test Case 1 — 2-Qubit Ansatz and Non-Degenerate Integer Hamiltonian

**Objective**  
Find the minimum eigenvalue of the Hamiltonian:
\begin{equation*}
H = Z \otimes Z + 2 \, X \otimes X
\end{equation*}

**Observable**  
This Hamiltonian mixes $Z$ and $X$ interactions across two qubits.

**Eigenvalues and Eigenvectors**  
After exact diagonalization, the eigenvalues and corresponding eigenvectors are:

- Eigenvalue: $-3$
\begin{equation*}
\ket{\psi_0} = \frac{1}{\sqrt{2}} \left( \ket{01} + \ket{10} \right)
\end{equation*}
- Eigenvalue: $-1$
\begin{equation*}
\ket{\psi_1} = \frac{1}{\sqrt{2}} \left( \ket{00} + \ket{11} \right)
\end{equation*}
- Eigenvalue: $+1$
\begin{equation*}
\ket{\psi_2} = \frac{1}{\sqrt{2}} \left( \ket{00} - \ket{11} \right)
\end{equation*}
- Eigenvalue: $+3$
\begin{equation*}
\ket{\psi_3} = \frac{1}{\sqrt{2}} \left( \ket{01} - \ket{10} \right)
\end{equation*}

**Expected Results**  
- Minimal eigenvalue: $-3$,
- The ground state is **non-degenerate** (unique),
- The optimizer should converge to a variational state close to:
\begin{equation*}
\ket{\psi_0} = \frac{1}{\sqrt{2}} \left( \ket{01} + \ket{10} \right)
\end{equation*}
achieving an expectation value near $-3$.


In [ ]:
from qiskit import QuantumCircuit
from qiskit.circuit import Parameter
from qiskit.quantum_info import SparsePauliOp, Statevector
import numpy as np

# === Step 1: Define manual 2-parameter ansatz ===
theta0 = Parameter("θ0")
theta1 = Parameter("θ1")

ansatz = QuantumCircuit(2)
ansatz.ry(theta0, 0)
ansatz.cx(1,0)
ansatz.ry(theta1, 1)
ansatz.cx(0, 1)

# === Step 2: Define observable (Hamiltonian) ===
observable = SparsePauliOp.from_list([
    ("ZZ", 1.0),
    ("XX", 2.0)
])

# === Step 3: Setup QSR ===
active_params = [theta0, theta1]
S_estimated = estimate_bandwidth_from_circuit(ansatz)
print(f"Estimated maximum Fourier harmonic S = {S_estimated}")

# === Step 4: Build sampling grid ===
num_points_per_param = 2 * S_estimated + 1
theta_ranges = [np.linspace(0, 2*np.pi, num_points_per_param, endpoint=False) for _ in active_params]
theta_mesh = np.meshgrid(*theta_ranges, indexing='ij')
theta_points = np.stack([mesh.flatten() for mesh in theta_mesh], axis=-1)

print(f"Total number of quantum circuit evaluations = {len(theta_points)}")

# === Step 5: Build parameter value arrays ===
theta_param_list = [theta_vector for theta_vector in theta_points]

# === Step 6: Sample expectation values ===
h_samples = sampler_qsr(theta_param_list, ansatz, observable, estimator)
h_samples = np.array(h_samples)

# === Step 7: Solve Fourier regression ===
coefficients, all_freqs = solve_fourier_coefficients(theta_points, h_samples, S_estimated)

# === Step 8: Minimize ===
theta_min, h_min = find_global_minimum(coefficients, all_freqs)
print(f"\nMinimal eigenvalue found via QSR: {h_min:.6f}")
print(f"Found at θ = {theta_min} radians")

# === Step 9: Final statevector ===
param_dict_opt = {param: theta_min[i] for i, param in enumerate(active_params)}
optimized_circuit = ansatz.assign_parameters(param_dict_opt)
final_state = Statevector(optimized_circuit)

# === Step 10: Print final statevector ===
terms = []
for idx, amplitude in enumerate(final_state.data):
    if np.abs(amplitude) > 1e-3:
        bstr = format(idx, '02b')  # for 2 qubits
        terms.append(f"({amplitude:.5f})|{bstr}⟩")

print("\nFinal statevector (QSR):", " + ".join(terms))


[DEBUG] Rotations: 2, Entanglement: True
Estimated maximum Fourier harmonic S = 4
Total number of quantum circuit evaluations = 81

Minimal eigenvalue found via QSR: -3.000000
Found at θ = [4.71238896 3.14159265] radians

Final statevector (QSR): (0.70711+0.00000j)|01⟩ + (-0.70711+0.00000j)|10⟩


: 